In [1]:
import torchhd
from torchhd.datasets import AirfoilSelfNoise
from torchhd import embeddings
import level

/Users/mikeheddes/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchmetrics
from tqdm import tqdm

In [3]:
DIMENSIONS = 10_000

In [4]:
class RegHD(nn.Module):
    def __init__(self, num_features, requires_grad=False) -> None:
        super(RegHD, self).__init__()
        
        self.num_features = num_features
        weight = torch.zeros(1, num_features)
        self.weight = nn.Parameter(weight, requires_grad)
    
    @torch.no_grad()
    def model_update(self, input, target, lr=0.00001):
        error = target - F.linear(input, self.weight)
        update = torch.sum(lr * error * input, dim=0)
        self.weight.add_(update)
        
    def forward(self, input):
        return F.linear(input, self.weight)

In [111]:
class LevelHV(nn.Module):
    def __init__(self, num_features, num_levels=100, randomness=0.0, requires_grad=False) -> None:
        super(LevelHV, self).__init__()

        self.num_levels = num_levels
        self.num_features = num_features

        self.levels = embeddings.Level(self.num_levels, num_features, low=0, high=1, randomness=randomness)
        self.memory = self.levels.weight

        weight = torch.zeros(1, num_features)
        self.weight = nn.Parameter(weight, requires_grad)
    
    @torch.no_grad()
    def model_update(self, input, target):
        target_vec = self.levels(target)
        input_target_pairs = torchhd.hash_table(input, target_vec)
        self.weight.add_(input_target_pairs)
        
    def forward(self, input):
        target_vec = torchhd.bind(self.weight, torchhd.inverse(input))
        memory_sim = torchhd.dot_similarity(target_vec, self.memory)
        level_idx = torch.argmax(memory_sim, dim=-1)
        return (level_idx / (self.num_levels - 1)) * (self.levels.high - self.levels.low) + self.levels.low

In [6]:
dataset = AirfoilSelfNoise('data', download=True)

DATA_STD = dataset.data.std(0)
DATA_MEAN = dataset.data.mean(0)
TARGET_STD = dataset.targets.std(0)
TARGET_MEAN = dataset.targets.mean(0)
DATA_MIN = dataset.data.min(0).values
DATA_MAX = dataset.data.max(0).values
TARGET_MIN = dataset.targets.min(0).values
TARGET_MAX = dataset.targets.max(0).values

Files already downloaded and verified


In [7]:
class NormalTransform:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, input):
        return (input - self.mean) / self.std

class MinMaxTransform:
    def __init__(self, min, max):
        self.min = min
        self.max = max

    def __call__(self, input):
        return (input - self.min) / (self.max - self.min)

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_data, batch_size=12, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=12)

In [8]:
item, label = next(iter(train_dataloader))
print(item)
print(label)

tensor([[6.3000e+02, 0.0000e+00, 2.2860e-01, 3.1700e+01, 2.7238e-03],
        [2.5000e+02, 1.5400e+01, 5.0800e-02, 3.1700e+01, 2.8985e-02],
        [1.2500e+03, 6.7000e+00, 1.0160e-01, 5.5500e+01, 5.2139e-03],
        [1.6000e+03, 1.9700e+01, 5.0800e-02, 7.1300e+01, 3.4118e-02],
        [1.0000e+03, 1.7400e+01, 2.5400e-02, 5.5500e+01, 1.6571e-02],
        [1.0000e+03, 1.7400e+01, 2.5400e-02, 3.9600e+01, 1.7221e-02],
        [5.0000e+03, 4.0000e+00, 3.0480e-01, 3.9600e+01, 5.7964e-03],
        [1.6000e+03, 0.0000e+00, 2.2860e-01, 3.9600e+01, 2.5351e-03],
        [4.0000e+03, 4.0000e+00, 2.2860e-01, 7.1300e+01, 4.0060e-03],
        [6.3000e+02, 2.0000e+00, 2.2860e-01, 5.5500e+01, 3.1352e-03],
        [6.3000e+02, 2.2200e+01, 2.5400e-02, 7.1300e+01, 2.1418e-02],
        [5.0000e+03, 4.0000e+00, 2.2860e-01, 3.1700e+01, 5.0907e-03]])
tensor([129.4650, 121.2250, 136.8830, 120.5750, 126.0010, 138.2740, 113.7330,
        127.0950, 121.7280, 126.8420, 120.6570, 115.9690])


In [34]:
encoder = embeddings.Projection(5, DIMENSIONS)

In [98]:
model = RegHD(DIMENSIONS)

dataset.transform = NormalTransform(DATA_MEAN, DATA_STD)
# dataset.transform = MinMaxTransform(DATA_MIN, DATA_MAX)
dataset.target_transform = NormalTransform(TARGET_MEAN, TARGET_STD)

In [99]:
with torch.no_grad():
    for epoch in range(1):
        for samples, labels in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
            samples_hv = encoder(samples).sign()
            model.model_update(samples_hv, labels.unsqueeze(-1))

Epoch 1: 100%|██████████| 88/88 [00:00<00:00, 2426.33it/s]


In [100]:
mse = torchmetrics.MeanSquaredError()

with torch.no_grad():
    for samples, labels in tqdm(test_dataloader, desc="Testing"):
        samples_hv = encoder(samples).sign()
        predictions = model(samples_hv)
        predictions = predictions * TARGET_STD + TARGET_MEAN
        labels = labels * TARGET_STD + TARGET_MEAN
        mse.update(predictions.cpu(), labels.unsqueeze(-1))

print(f"Testing mean squared error of {(mse.compute().item()):.3f}")

Testing: 100%|██████████| 38/38 [00:00<00:00, 2664.79it/s]

Testing mean squared error of 19.513


In [130]:
model = LevelHV(DIMENSIONS, randomness=0.1)

# dataset.transform = NormalTransform(DATA_MEAN, DATA_STD)
dataset.transform = MinMaxTransform(DATA_MIN, DATA_MAX)
# dataset.target_transform = NormalTransform(TARGET_MEAN, TARGET_STD)
dataset.target_transform = MinMaxTransform(TARGET_MIN, TARGET_MAX)

In [131]:
with torch.no_grad():
    for epoch in range(1):
        for samples, labels in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
            samples_hv = encoder(samples).sign()
            model.model_update(samples_hv, labels)

Epoch 1: 100%|██████████| 88/88 [00:00<00:00, 1958.60it/s]


In [132]:
mse = torchmetrics.MeanSquaredError()

with torch.no_grad():
    for samples, labels in tqdm(test_dataloader, desc="Testing"):
        samples_hv = encoder(samples).sign()
        predictions = model(samples_hv)
        predictions = predictions * (TARGET_MAX - TARGET_MIN) + TARGET_MIN
        labels = labels * (TARGET_MAX - TARGET_MIN) + TARGET_MIN
        mse.update(predictions.cpu(), labels)

print(f"Testing mean squared error of {(mse.compute().item()):.3f}")

Testing: 100%|██████████| 38/38 [00:00<00:00, 1266.75it/s]

Testing mean squared error of 44.254
